In [1]:
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Dense, LeakyReLU, Input
from keras.models import Sequential, Model
from keras.datasets import mnist
from keras.optimizers import Adam

In [2]:
((trainx, trainy), (testx, testy)) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
print(trainx.shape, trainy.shape, testx.shape, testy.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [4]:
def drawImg(x, title):
  x = x.reshape(28, 28, 1)
  plt.imshow(x, cmap='gray')
  plt.title(title)
  plt.show()

# drawImg(trainx[0], trainy[0])

In [5]:
# mormalize the values [-1, 1]

mean = 255.0/2
std = 255.0/2

trainx = trainx.astype('float')
trainx = (trainx - mean) / std
print(np.min(trainx))
print(np.max(trainx))
print(trainx.shape)

-1.0
1.0
(60000, 28, 28)


In [6]:
TOTAL_EPOCHS = 50
BATCH_SIZE = 256
NO_OF_BATCHES = int(trainx.shape[0] / BATCH_SIZE)
HALF_BATCH = 128
NOISE_DIM = 100 # UPSAMPLE UPTO 784 DIMENSIONS
ADAM = Adam(learning_rate=2e-4, beta_1=0.5)

# Generator
- input_noise = 100 dimensional
- output = 784 dimensional

In [7]:
generator = Sequential()
generator.add(Dense(256, input_dim=(NOISE_DIM)))
generator.add(LeakyReLU(alpha=0.2))
generator.add(Dense(512))
generator.add(LeakyReLU(alpha=0.2))
generator.add(Dense(1024))
generator.add(LeakyReLU(alpha=0.2))
generator.add(Dense(784, activation='tanh'))

generator.compile(loss='binary_crossentropy', optimizer=ADAM)
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               25856     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 1024)              525312    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 784)               8

In [8]:
discriminator = Sequential()
discriminator.add(Dense(512, input_shape=(784,)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss="binary_crossentropy", optimizer='adam')
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 512)               401920    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533505 (2.04 MB)
Trainable params: 533505 (2.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Binding Generator and Discriminator in a single model

In [9]:
discriminator.trainable = False

gan_input = Input(shape=(NOISE_DIM))
generated_img = generator(gan_input)
gan_output = discriminator(generated_img)

model = Model(gan_input, gan_output)
model.compile(loss='binary_crossentropy', optimizer=ADAM)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 784)               1486352   
                                                                 
 sequential_1 (Sequential)   (None, 1)                 533505    
                                                                 
Total params: 2019857 (7.71 MB)
Trainable params: 1486352 (5.67 MB)
Non-trainable params: 533505 (2.04 MB)
_________________________________________________________________


In [10]:
trainx = trainx.reshape((-1, 784))
print(trainx.shape)

(60000, 784)


In [11]:
def save_imgs(epoch, samples=100):

  noise = np.random.normal(0, 1, size=(samples, NOISE_DIM))
  generated_imgs = generator.predict(noise)
  generated_imgs = generated_imgs.reshape(samples, 28, 28)

  plt.figure(figsize=(10,10))
  for i in range(samples):
    plt.subplot(10, 10, i + 1)
    plt.imshow(generated_imgs[i], cmap='gray', interpolation='nearest')
    plt.axis('off')

  plt.tight_layout()
  plt.savefig('images/gen_output_epoch_{0}.png'.format(epoch + 1))
  plt.show()

In [12]:
# Training Loop
%%capture
for epoch in range(TOTAL_EPOCHS):
  epoch_d_loss = 0.0
  epoch_g_loss = 0.0

  for step in range(NO_OF_BATCHES):
    # step-1 Discriminator
    # 50% Real Data + Fake Data

    # Real Data
    idx = np.random.randint(0, trainx.shape[0], HALF_BATCH)
    real_imgs = trainx[idx]

    # Fake Data
    noise = np.random.normal(0, 1, size=(HALF_BATCH, NOISE_DIM))
    fake_imgs = generator.predict(noise) # Forward

    # create labels for fake and real
    real_y = np.ones((HALF_BATCH, 1))*0.9
    fake_y = np.zeros((HALF_BATCH, 1))

    # Train our discriminator
    d_loss_real = discriminator.train_on_batch(real_imgs, real_y)
    d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_y)
    d_loss = 0.5 * d_loss_real + 0.5 * d_loss_fake
    epoch_d_loss += d_loss

    # step-2 Train Generator(Considering Frozen Discriminator)
    noise = np.random.normal(0, 1, size=(BATCH_SIZE, NOISE_DIM))
    ground_truth_y = np.ones((BATCH_SIZE, 1))
    g_loss = model.train_on_batch(noise, ground_truth_y)
    epoch_g_loss += g_loss

  print('Epoch: %d, Discriminator Loss: %.4f, Generator Loss: %.4f'%(epoch, epoch_d_loss/NO_OF_BATCHES, epoch_g_loss/NO_OF_BATCHES))

  if (epoch + 1) % 5 == 0:
    generator.save('gan_model/generator_'+f'{epoch + 1}'+'.h5')
    save_imgs(epoch)

In [13]:
!zip -r /content/images.zip /content/images
from google.colab import files
files.download('images.zip')

  adding: content/images/ (stored 0%)
  adding: content/images/gen_output_epoch_15.png (deflated 9%)
  adding: content/images/gen_output_epoch_45.png (deflated 11%)
  adding: content/images/gen_output_epoch_25.png (deflated 9%)
  adding: content/images/gen_output_epoch_30.png (deflated 10%)
  adding: content/images/gen_output_epoch_5.png (deflated 9%)
  adding: content/images/gen_output_epoch_10.png (deflated 9%)
  adding: content/images/gen_output_epoch_20.png (deflated 10%)
  adding: content/images/gen_output_epoch_35.png (deflated 10%)
  adding: content/images/gen_output_epoch_50.png (deflated 11%)
  adding: content/images/gen_output_epoch_40.png (deflated 11%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')